# uTensor End-to-End Workflow - Preview

## Imports

In [1]:
from utensor_cgen.api.export import tflm_keras_export

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

tf.keras.backend.clear_session()
print("TensorFlow Version: " + tf.__version__)

TensorFlow Version: 2.1.0


## Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

[tf.keras.datasets.mnist](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data) returns: ***Tuple of Numpy arrays: (x_train, y_train), (x_test, y_test).***

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [4]:
# Shuffling the dataset and forming batches
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Defining the Model

In [5]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(8, 3, activation='relu')
    self.pool = MaxPooling2D()
    self.flatten = Flatten()
    self.d1 = Dense(16, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x0 = self.pool(x)
    x1 = self.conv1(x0)
    x2 = self.pool(x1)
    x3 = self.flatten(x2)
    x4 = self.d1(x3)
    return self.d2(x4)

model = MyModel()

## Loss Functions, and Optimisers

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Defining Training and Testing Steps

#### Training Step

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

#### Model Test Step

In [9]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## Training Loop

In [10]:
EPOCHS = 15

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.4687080681324005, Accuracy: 86.461669921875, Test Loss: 0.20880873501300812, Test Accuracy: 93.45999908447266
Epoch 2, Loss: 0.17733371257781982, Accuracy: 94.58666229248047, Test Loss: 0.14247198402881622, Test Accuracy: 95.41000366210938
Epoch 3, Loss: 0.13928145170211792, Accuracy: 95.74833679199219, Test Loss: 0.11060388386249542, Test Accuracy: 96.47000122070312
Epoch 4, Loss: 0.11995214223861694, Accuracy: 96.35000610351562, Test Loss: 0.1065719872713089, Test Accuracy: 96.7200012

## Exporting to uTensor

A sample generator for the offline-quantizer to estimate the optimal quantization ranges for all activations.

In [11]:
# representative data function
num_calibration_steps = 128
calibration_dtype = tf.float32

def representative_dataset_gen():
    for _ in range(num_calibration_steps):
        rand_idx = np.random.randint(0, x_test.shape[0]-1)
        sample = x_test[rand_idx]
        sample = sample[tf.newaxis, ...]
        sample = tf.cast(sample, dtype=calibration_dtype)
        yield [sample]

One-liner uTensor export API

In [12]:
tflm_keras_export(
    model,
    representive_dataset=representative_dataset_gen,
    model_name='my_model',
    target='utensor',
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /var/folders/9x/bvz7kpds5c99st2593g_w9hm0000gn/T/utensor_erb1n8ng/saved_model/assets
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING quantize.py <module> @ 12] trying to import deprecated quantization transformer
[INFO transformer.py transform @ 23] Transforming graph: my_model
[INFO transformer.py transform @ 24] Transform pipeline: dropout(name_pattern=r'(dropout[_\w\d]*)/.*') -> linear_reorder -> inline -> biasAdd -> remove_id_op -> fake_gather_v2 -> refcnt
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING b

In [13]:
!pwd

/Users/neiltan/code/ut/utensor-helloworld
